In [1]:
def composites(wave_ds,precip_ds):    
    wavedict = {}
    
    time_phase = wave_ds["time"].astype("datetime64[D]")
    time_precip = precip_ds["time"].astype("datetime64[D]")

    phases = wave_ds.groupby("phase").groups

    for phase in phases:
        phase_indices = np.where(phase == wave_ds["phase"])[0]
        precip_values = [
            precip_ds.where(time_phase[i] == time_precip).dropna(dim = "time").values[0]
            for i in phase_indices
        ]
        precip_time = [
            precip_ds["time"].where(time_phase[i] == time_precip).dropna(dim = "time").values[0]
            for i in phase_indices
        ]
        wavedict[str(int(phase))] = [precip_time, precip_values]
#         wavedict["time-"+str(int(phase))] = precip_time
    return(wavedict)

def is_seas(month, seas):
    if seas=='jja':
        return (month >= 6) & (month <= 8)
    elif seas=='djf':
        return (month >= 12) | (month <= 2)
    elif seas=='son':
        return (month >= 9) & (month <= 11)
    elif seas=='mam':
        return (month >= 3) & (month <= 5)
        
def placehold(array,fillval):
    if len(array)==0:
        return(fillval)
    else: return(array)

In [2]:
import xarray as xr
import pandas as pd
import numpy as np

seas = 'djf'
if seas=='mam':
    th = 5.98715
    seasname = "SHAutumn"
elif seas=='djf':
    th = 11.46076
    seasname = "SHSummer"

In [3]:
diri = "/Users/fadhlilrizki/Documents/PhD/thesis/dataout/local_wave_phase/"

dirinc ="/Users/fadhlilrizki/Documents/PhD/thesis/dataout/"

diro = "/Users/fadhlilrizki/Documents/PhD/thesis/dataout/local_wave_phase/combined_wave/"

In [4]:
regions = ["TopEnd","CapeYork","Kimberley","Queensland"]
latSs   = [-17, -16, -20, -25]
latNs   = [-10, -10, -13.5, -20]
lonRs   = [137, 146, 128, 145]
lonLs   = [129, 142, 120, 138]

code = 3 #regioncode
if code==0:
    region ="TE"
elif code==1:
    region="CY"
elif code==2:
    region="KY"
elif code==3:
    region="QL"

In [5]:
fw_1 = "local_er_phase_"+seasname+".133.nc"
fw_2 = "local_kelvin_phase_"+seasname+".133.nc"
fw_3 = "local_mrg_phase_"+seasname+".133.nc"
fw_4 = "local_td_phase_"+seasname+".133.nc"
fw_5 = "local_mjo_phase_"+seasname+".133.nc"

fe_1 = "er_wave_extreme_"+region+"_"+seas+".nc"
fe_2 = "kelvin_wave_extreme_"+region+"_"+seas+".nc"
fe_3 = "mrg_wave_extreme_"+region+"_"+seas+".nc"
fe_4 = "td_wave_extreme_"+region+"_"+seas+".nc"
fe_5 = "mjo_wave_extreme_"+region+"_"+seas+".nc"


In [6]:
dw_1 = xr.open_dataset(diri+fw_1)
dw_2 = xr.open_dataset(diri+fw_2)
dw_3 = xr.open_dataset(diri+fw_3)
dw_4 = xr.open_dataset(diri+fw_4)
dw_5 = xr.open_dataset(diri+fw_5)

# dprec = xr.open_dataset("/Users/fadhlilrizki/Documents/PhD/thesis/data/agcd_v1_precip_calib_r025_daily_1981-2020.masked.nc")

In [7]:
de_1 = xr.open_dataset(dirinc+fe_1)
de_2 = xr.open_dataset(dirinc+fe_2)
de_3 = xr.open_dataset(dirinc+fe_3)
de_4 = xr.open_dataset(dirinc+fe_4)
de_5 = xr.open_dataset(dirinc+fe_5)

# dprec = xr.open_dataset("/Users/fadhlilrizki/Documents/PhD/thesis/data/agcd_v1_precip_calib_r025_daily_1981-2020.masked.nc")

In [8]:
erp  = de_1.composite.values
klvp = de_2.composite.values
mrgp = de_3.composite.values
tdp  = de_4.composite.values
mjop = de_5.composite.values

In [9]:
dw_1 = dw_1.sel(time=slice("1981-01-01","2018-12-31"))
dw_2 = dw_2.sel(time=slice("1981-01-01","2018-12-31"))
dw_3 = dw_3.sel(time=slice("1981-01-01","2018-12-31"))
dw_4 = dw_4.sel(time=slice("1981-01-01","2018-12-31"))
dw_5 = dw_5.sel(time=slice("1981-01-01","2018-12-31"))

# dprec = dprec.sel(time=slice("1981-01-01","2018-12-31"),
#                   lon=slice(lonLs[code],lonRs[code]), 
#                   lat=slice(latSs[code],latNs[code])
#                  )

# prec = dprec.precip
# prec

In [10]:
# weights = np.cos(np.deg2rad(prec.lat))
# weights.name = "weights"

# pr_weighted = prec.weighted(weights)
# praave = pr_weighted.mean(("lon", "lat"))

# praave

In [11]:
er_active     = xr.where(dw_1["amp"] >= 1, dw_1, np.nan).dropna(dim="time")
kelvin_active = xr.where(dw_2["amp"] >= 1, dw_2, np.nan).dropna(dim="time")
mrg_active    = xr.where(dw_3["amp"] >= 1, dw_3, np.nan).dropna(dim="time")
td_active     = xr.where(dw_4["amp"] >= 1, dw_4, np.nan).dropna(dim="time")
mjo_active    = xr.where(dw_5["amp"] >= 1, dw_5, np.nan).dropna(dim="time")


In [12]:
# er_active.to_dataframe().to_csv('er_act.csv')

In [13]:
tc   = pd.read_csv('/Users/fadhlilrizki/Documents/PhD/thesis/data/TC_dates.csv',
                   usecols=["0"], parse_dates=["0"])

tc_da = xr.DataArray(tc)
er_active         = xr.where(~er_active['time'].isin(tc_da), er_active, np.nan).dropna(dim='time')
mrg_active        = xr.where(~mrg_active['time'].isin(tc_da), mrg_active, np.nan).dropna(dim='time')
kelvin_active     = xr.where(~kelvin_active['time'].isin(tc_da), kelvin_active, np.nan).dropna(dim='time')
td_active         = xr.where(~td_active['time'].isin(tc_da), td_active, np.nan).dropna(dim='time')
mjo_active        = xr.where(~mjo_active['time'].isin(tc_da), mjo_active, np.nan).dropna(dim='time')

er_active

<xarray.Dataset>
Dimensions:  (time: 2099)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2018-12-31
Data variables:
    x        (time) float32 1.602 1.671 1.368 0.9369 ... -1.516 -1.504 -1.202
    y        (time) float32 0.2745 0.01123 -0.9718 ... -0.3186 0.4537 1.255
    amp      (time) float32 1.626 1.671 1.678 1.72 ... 1.559 1.549 1.571 1.738
    phase    (time) float64 1.0 1.0 2.0 2.0 3.0 3.0 ... 4.0 4.0 4.0 5.0 5.0 6.0

In [14]:
er_size = er_active.groupby("phase").groups
key_to_value_lengths = {k:len(v) for k, v in er_size.items()}
key_to_value_lengths

{1.0: 268,
 2.0: 259,
 3.0: 292,
 4.0: 292,
 5.0: 258,
 6.0: 251,
 7.0: 237,
 8.0: 242}

In [15]:
er_comp  = {}
klv_comp = {}
mrg_comp = {}
td_comp  = {}
mjo_comp = {}

In [16]:
th

11.46076

In [17]:
er_comp = erp
# er_comp_mean = dict((k, np.mean(v[1])/prob_season) for k, v in er_comp.items())

mrg_comp = mrgp
# mrg_comp_mean = dict((k, np.mean(v[1])/prob_season) for k, v in mrg_comp.items())

kelvin_comp = klvp
# kelvin_comp_mean = dict((k, np.mean(v[1])/prob_season) for k, v in kelvin_comp.items())

td_comp = tdp
# td_comp_mean = dict((k, np.mean(v[1])/prob_season) for k, v in td_comp.items())

mjo_comp = mjop
# mjo_comp_mean = dict((k, np.mean(v[1])/prob_season) for k, v in mjo_comp.items())

In [18]:
er_comp
# er_comp_mean
# mrg_comp_mean

array([0.58158076, 1.2035803 , 0.978596  , 1.2454857 , 1.3089324 ,
       1.3454366 , 0.87687   , 1.1807852 ], dtype=float32)

In [19]:
mrgp

array([0.55507   , 0.82280964, 0.89192355, 1.326499  , 1.4746923 ,
       0.950388  , 0.80460584, 0.80460584], dtype=float32)

In [20]:

er_wets     = [int(k+1) for k,v in enumerate(erp) if v>=1.25]
kelvin_wets = [int(k+1) for k,v in enumerate(klvp) if v>=1.25]
mrg_wets    = [int(k+1) for k,v in enumerate(mrgp) if v>=1.25]
td_wets     = [int(k+1) for k,v in enumerate(tdp) if v>=1.25]
mjo_wets    = [int(k+1) for k,v in enumerate(mjop) if v>=1.25]

er_drys     = [int(k+1) for k,v in enumerate(erp) if v<=0.8]
kelvin_drys = [int(k+1) for k,v in enumerate(klvp) if v<=0.8]
mrg_drys    = [int(k+1) for k,v in enumerate(mrgp) if v<=0.8]
td_drys     = [int(k+1) for k,v in enumerate(tdp) if v<=0.8]
mjo_drys    = [int(k+1) for k,v in enumerate(mjop) if v<=0.8]

with open('phases_'+region+'_'+seas+'.txt', 'w') as f:
    f.write(str(["er-wet",er_wets,"dry", er_drys]))
    f.write("\n"+str(["kelvin-wet",kelvin_wets,"dry", kelvin_drys]))
    f.write("\n"+str(["mrg-wet",mrg_wets,"dry", mrg_drys]))
    f.write("\n"+str(["td-wet",td_wets,"dry", td_drys]))
    f.write("\n"+str(["mjo-wet",mjo_wets,"dry", mjo_drys]))
# er_wets

In [21]:
print(seas)
print(str(["er-wet",er_wets,"dry", er_drys]))
print("\n"+str(["kelvin-wet",kelvin_wets,"dry", kelvin_drys]))
print("\n"+str(["mrg-wet",mrg_wets,"dry", mrg_drys]))
print("\n"+str(["td-wet",td_wets,"dry", td_drys]))
print("\n"+str(["mjo-wet",mjo_wets,"dry", mjo_drys]))

djf
['er-wet', [5, 6], 'dry', [1]]

['kelvin-wet', [1, 7], 'dry', [4, 6]]

['mrg-wet', [4, 5], 'dry', [1]]

['td-wet', [2, 3, 4], 'dry', [6, 7, 8]]

['mjo-wet', [5, 6, 7], 'dry', [1, 2, 3, 4]]


In [22]:
if seas=='djf':
    er_wet        = xr.where((er_active["phase"]==5) | (er_active["phase"]==6), er_active, np.nan).dropna(dim="time")
    kelvin_wet    = xr.where((kelvin_active["phase"]==7) | (kelvin_active["phase"]==1) , kelvin_active, np.nan).dropna(dim="time") 
    mrg_wet       = xr.where((mrg_active["phase"]==5) | (mrg_active["phase"]==4), mrg_active, np.nan).dropna(dim="time")
    td_wet        = xr.where((td_active["phase"]==2) | (td_active["phase"]==3)| (td_active["phase"]==4) , td_active, np.nan).dropna(dim="time")
    mjo_wet       = xr.where(((mjo_active["phase"]>=5) & (mjo_active["phase"]<=7)), mjo_active, np.nan).dropna(dim="time")
elif seas=='mam':
    er_wet        = xr.where((er_active["phase"]==4) | (er_active["phase"]==6) , er_active, np.nan).dropna(dim="time") #gaada
    kelvin_wet    = xr.where((kelvin_active["phase"]==2) | (kelvin_active["phase"]==4) | (kelvin_active["phase"]==8), kelvin_active, np.nan).dropna(dim="time")
    mrg_wet       = xr.where((mrg_active["phase"]==1) | (mrg_active["phase"]==7) | (mrg_active["phase"]>=2) & (mrg_active["phase"]<=5), mrg_active, np.nan).dropna(dim="time")
    td_wet        = xr.where((td_active["phase"]==2) | (td_active["phase"]==4) | (td_active["phase"]==5), td_active, np.nan).dropna(dim="time")
    mjo_wet       = xr.where((mjo_active["phase"]>=5) & (mjo_active["phase"]<=7), mjo_active, np.nan).dropna(dim="time")
    

In [23]:
if seas=='djf':
    er_dry        = xr.where((er_active["phase"]==1) , er_active, np.nan).dropna(dim="time")
    kelvin_dry    = xr.where((kelvin_active["phase"]==6) | (kelvin_active["phase"]==4) , kelvin_active, np.nan).dropna(dim="time") #gaada
    mrg_dry       = xr.where((mrg_active["phase"]==1), mrg_active, np.nan).dropna(dim="time")
    td_dry        = xr.where((td_active["phase"]==6) | (td_active["phase"]==7) | (td_active["phase"]==8)  , td_active, np.nan).dropna(dim="time")
    mjo_dry       = xr.where((mjo_active["phase"]<=4), mjo_active, np.nan).dropna(dim="time")
elif seas=='mam':
    er_dry        = xr.where((er_active["phase"]<=3) | (er_active["phase"]>=7), er_active, np.nan).dropna(dim="time")
    kelvin_dry    = xr.where((kelvin_active["phase"]==1) | (kelvin_active["phase"]==3) , kelvin_active, np.nan).dropna(dim="time")
    mrg_dry       = xr.where((mrg_active["phase"]==6) | (mrg_active["phase"]==8) , mrg_active, np.nan).dropna(dim="time") #gaada
    td_dry        = xr.where((td_active["phase"]==6) | (td_active["phase"]==8), td_active, np.nan).dropna(dim="time")
    mjo_dry       = xr.where((mjo_active["phase"]==1) | (mjo_active["phase"]==3) | (mjo_active["phase"]==4)  , mjo_active, np.nan).dropna(dim="time")


In [24]:
er_wet

<xarray.Dataset>
Dimensions:  (time: 509)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-10 1981-01-11 ... 2018-12-31
Data variables:
    x        (time) float32 -1.201 -1.025 -2.283 -2.492 ... -1.516 -1.504 -1.202
    y        (time) float32 0.128 0.8126 -0.9065 -0.19 ... -0.3186 0.4537 1.255
    amp      (time) float32 1.207 1.308 2.456 2.499 ... 2.437 1.549 1.571 1.738
    phase    (time) float64 5.0 6.0 5.0 5.0 5.0 6.0 ... 6.0 5.0 5.0 5.0 5.0 6.0

In [25]:
er_wet

<xarray.Dataset>
Dimensions:  (time: 509)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-10 1981-01-11 ... 2018-12-31
Data variables:
    x        (time) float32 -1.201 -1.025 -2.283 -2.492 ... -1.516 -1.504 -1.202
    y        (time) float32 0.128 0.8126 -0.9065 -0.19 ... -0.3186 0.4537 1.255
    amp      (time) float32 1.207 1.308 2.456 2.499 ... 2.437 1.549 1.571 1.738
    phase    (time) float64 5.0 6.0 5.0 5.0 5.0 6.0 ... 6.0 5.0 5.0 5.0 5.0 6.0

In [26]:
#ER AND KELVIN
er_er_wetwet  = er_wet["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")
er_er_drydry  = er_dry["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")

klv_klv_wetwet  = kelvin_wet["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")
klv_klv_drydry  = kelvin_dry["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")

mrg_mrg_wetwet  = mrg_wet["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_mrg_drydry  = mrg_dry["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")

td_td_wetwet  = td_wet["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")
td_td_drydry  = td_dry["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")

mjo_mjo_wetwet  = mjo_wet["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")
mjo_mjo_drydry  = mjo_dry["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")


#ER kelvin
er_klv_wetwet = kelvin_wet["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_klv_drywet = kelvin_wet["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_klv_drydry = kelvin_dry["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_klv_wetdry = kelvin_dry["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

#MRG and KELVIN

mrg_klv_wetwet = kelvin_wet["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_klv_drywet = kelvin_wet["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_klv_drydry = kelvin_dry["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_klv_wetdry = kelvin_dry["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

#TD and Kelvin

td_klv_wetwet  = kelvin_wet["time"].where(td_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_klv_drywet  = kelvin_wet["time"].where(td_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_klv_drydry  = kelvin_dry["time"].where(td_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_klv_wetdry  = kelvin_dry["time"].where(td_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

#MJO and Kelvin

mjo_klv_wetwet  = kelvin_wet["time"].where(mjo_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mjo_klv_drywet  = kelvin_wet["time"].where(mjo_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mjo_klv_drydry  = kelvin_dry["time"].where(mjo_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mjo_klv_wetdry  = kelvin_dry["time"].where(mjo_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")



In [27]:
#ER AND MRG
er_mrg_wetwet = mrg_wet["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mrg_drywet = mrg_wet["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mrg_drydry = mrg_dry["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mrg_wetdry = mrg_dry["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [28]:
#ER AND TD
er_td_wetwet = td_wet["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_td_drywet = td_wet["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_td_drydry = td_dry["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_td_wetdry = td_dry["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [29]:
#ER AND MJO
er_mjo_wetwet = mjo_wet["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mjo_drywet = mjo_wet["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mjo_drydry = mjo_dry["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mjo_wetdry = mjo_dry["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [30]:
#mrg AND MJO
mrg_mjo_wetwet = mjo_wet["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_mjo_drywet = mjo_wet["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_mjo_drydry = mjo_dry["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_mjo_wetdry = mjo_dry["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [31]:
#TD AND MJO
td_mjo_wetwet = mjo_wet["time"].where(td_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_mjo_drywet = mjo_wet["time"].where(td_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_mjo_drydry = mjo_dry["time"].where(td_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_mjo_wetdry = mjo_dry["time"].where(td_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [32]:
#MRG AND TD
mrg_td_wetwet = td_wet["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_td_drywet = td_wet["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_td_drydry = td_dry["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_td_wetdry = td_dry["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [33]:
er_er_wetwet

,date
time,
1981-01-10,19810110
1981-01-11,19810111
1981-01-24,19810124
1981-01-25,19810125
1981-01-26,19810126
...,...
2018-02-27,20180227
2018-02-28,20180228
2018-12-29,20181229


In [34]:
er_er_wetwet.to_csv(diro+"er_er_wetwet_"+region+"_"+seas+".csv", columns=['date'], index = False)
er_er_drydry.to_csv(diro+"er_er_drydry_"+region+"_"+seas+".csv", columns=['date'], index = False)

er_klv_wetwet.to_csv(diro+"er_klv_wetwet_"+region+"_"+seas+".csv", columns=['date'], index = False)
er_klv_wetdry.to_csv(diro+"er_klv_wetdry_"+region+"_"+seas+".csv", columns=['date'], index = False)
er_klv_drywet.to_csv(diro+"er_klv_drywet_"+region+"_"+seas+".csv", columns=['date'], index = False)
er_klv_drydry.to_csv(diro+"er_klv_drydry_"+region+"_"+seas+".csv", columns=['date'], index = False)

er_mrg_wetwet.to_csv(diro+"er_mrg_wetwet_"+region+"_"+seas+".csv", columns=['date'], index = False)
er_mrg_wetdry.to_csv(diro+"er_mrg_wetdry_"+region+"_"+seas+".csv", columns=['date'], index = False)
er_mrg_drywet.to_csv(diro+"er_mrg_drywet_"+region+"_"+seas+".csv", columns=['date'], index = False)
er_mrg_drydry.to_csv(diro+"er_mrg_drydry_"+region+"_"+seas+".csv", columns=['date'], index = False)

er_td_wetwet.to_csv(diro+"er_td_wetwet_"+region+"_"+seas+".csv", columns=['date'], index = False)
er_td_wetdry.to_csv(diro+"er_td_wetdry_"+region+"_"+seas+".csv", columns=['date'], index = False)
er_td_drywet.to_csv(diro+"er_td_drywet_"+region+"_"+seas+".csv", columns=['date'], index = False)
er_td_drydry.to_csv(diro+"er_td_drydry_"+region+"_"+seas+".csv", columns=['date'], index = False)

er_mjo_wetwet.to_csv(diro+"er_mjo_wetwet_"+region+"_"+seas+".csv", columns=['date'], index = False)
er_mjo_wetdry.to_csv(diro+"er_mjo_wetdry_"+region+"_"+seas+".csv", columns=['date'], index = False)
er_mjo_drywet.to_csv(diro+"er_mjo_drywet_"+region+"_"+seas+".csv", columns=['date'], index = False)
er_mjo_drydry.to_csv(diro+"er_mjo_drydry_"+region+"_"+seas+".csv", columns=['date'], index = False)

klv_klv_wetwet.to_csv(diro+"klv_klv_wetwet_"+region+"_"+seas+".csv", columns=['date'], index = False)
klv_klv_drydry.to_csv(diro+"klv_klv_drydry_"+region+"_"+seas+".csv", columns=['date'], index = False)

mrg_mrg_wetwet.to_csv(diro+"mrg_mrg_wetwet_"+region+"_"+seas+".csv", columns=['date'], index = False)
mrg_mrg_drydry.to_csv(diro+"mrg_mrg_drydry_"+region+"_"+seas+".csv", columns=['date'], index = False)

mrg_klv_wetwet.to_csv(diro+"mrg_klv_wetwet_"+region+"_"+seas+".csv", columns=['date'], index = False)
mrg_klv_wetdry.to_csv(diro+"mrg_klv_wetdry_"+region+"_"+seas+".csv", columns=['date'], index = False)
mrg_klv_drywet.to_csv(diro+"mrg_klv_drywet_"+region+"_"+seas+".csv", columns=['date'], index = False)
mrg_klv_drydry.to_csv(diro+"mrg_klv_drydry_"+region+"_"+seas+".csv", columns=['date'], index = False)

mrg_td_wetwet.to_csv(diro+"mrg_td_wetwet_"+region+"_"+seas+".csv", columns=['date'], index = False)
mrg_td_wetdry.to_csv(diro+"mrg_td_wetdry_"+region+"_"+seas+".csv", columns=['date'], index = False)
mrg_td_drywet.to_csv(diro+"mrg_td_drywet_"+region+"_"+seas+".csv", columns=['date'], index = False)
mrg_td_drydry.to_csv(diro+"mrg_td_drydry_"+region+"_"+seas+".csv", columns=['date'], index = False)

mrg_mjo_wetwet.to_csv(diro+"mrg_mjo_wetwet_"+region+"_"+seas+".csv", columns=['date'], index = False)
mrg_mjo_wetdry.to_csv(diro+"mrg_mjo_wetdry_"+region+"_"+seas+".csv", columns=['date'], index = False)
mrg_mjo_drywet.to_csv(diro+"mrg_mjo_drywet_"+region+"_"+seas+".csv", columns=['date'], index = False)
mrg_mjo_drydry.to_csv(diro+"mrg_mjo_drydry_"+region+"_"+seas+".csv", columns=['date'], index = False)

td_td_wetwet.to_csv(diro+"td_td_wetwet_"+region+"_"+seas+".csv", columns=['date'], index = False)
td_td_drydry.to_csv(diro+"td_td_drydry_"+region+"_"+seas+".csv", columns=['date'], index = False)

td_klv_wetwet.to_csv(diro+"td_klv_wetwet_"+region+"_"+seas+".csv", columns=['date'], index = False)
td_klv_wetdry.to_csv(diro+"td_klv_wetdry_"+region+"_"+seas+".csv", columns=['date'], index = False)
td_klv_drywet.to_csv(diro+"td_klv_drywet_"+region+"_"+seas+".csv", columns=['date'], index = False)
td_klv_drydry.to_csv(diro+"td_klv_drydry_"+region+"_"+seas+".csv", columns=['date'], index = False)

td_mjo_wetwet.to_csv(diro+"td_mjo_wetwet_"+region+"_"+seas+".csv", columns=['date'], index = False)
td_mjo_wetdry.to_csv(diro+"td_mjo_wetdry_"+region+"_"+seas+".csv", columns=['date'], index = False)
td_mjo_drywet.to_csv(diro+"td_mjo_drywet_"+region+"_"+seas+".csv", columns=['date'], index = False)
td_mjo_drydry.to_csv(diro+"td_mjo_drydry_"+region+"_"+seas+".csv", columns=['date'], index = False)



mjo_mjo_wetwet.to_csv(diro+"mjo_mjo_wetwet_"+region+"_"+seas+".csv", columns=['date'], index = False)
mjo_mjo_drydry.to_csv(diro+"mjo_mjo_drydry_"+region+"_"+seas+".csv", columns=['date'], index = False)

mjo_klv_wetwet.to_csv(diro+"mjo_klv_wetwet_"+region+"_"+seas+".csv", columns=['date'], index = False)
mjo_klv_wetdry.to_csv(diro+"mjo_klv_wetdry_"+region+"_"+seas+".csv", columns=['date'], index = False)
mjo_klv_drywet.to_csv(diro+"mjo_klv_drywet_"+region+"_"+seas+".csv", columns=['date'], index = False)
mjo_klv_drydry.to_csv(diro+"mjo_klv_drydry_"+region+"_"+seas+".csv", columns=['date'], index = False)


In [35]:
er_td_drydry

,date
time,
1981-01-01,19810101
1981-01-02,19810102
1981-12-09,19811209
1981-12-10,19811210
1981-12-15,19811215
...,...
2014-12-25,20141225
2014-12-26,20141226
2015-02-02,20150202


In [36]:
! ncl 'seas="{seas}"' code={code} composite_combined_area_wave_forpy.ncl

 Copyright (C) 1995-2019 - All Rights Reserved
 University Corporation for Atmospheric Research
 NCAR Command Language Version 6.6.2
 The use of this software is governed by a License Agreement.
 See http://www.ncl.ucar.edu/ for more details.

Variable: pr
Type: float
Total Size: 57580 bytes
            14395 values
Number of Dimensions: 1
Dimensions and sizes:	[time | 14395]
Coordinates: 
            time: [47847.375..62241.375]
Number Of Attributes: 14
  _FillValue :	1e+20
  coverage_content_type :	physicalMeasurement
  length_scale_for_analysis :	  80
  frequency :	daily
  number_of_stations_reporting :	6600
  analysis_version_number :	3.01
  cell_methods :	time: sum
  least_significant_digit :	3
  grid_mapping :	spatial_ref
  units :	mm
  long_name :	Daily precipitation
  standard_name :	lwe_thickness_of_precipitation_amount
  missing_value :	1e+20
  wgt_areaave_op_ncl :	Area Average


Variable: pr_Anom
Type: float
Total Size: 57580 bytes
            14395 values
Number of Dimensio

In [37]:
! ncl 'seas="{seas}"' code={code} composite_combined_area_wave_extremes_forpy.ncl

 Copyright (C) 1995-2019 - All Rights Reserved
 University Corporation for Atmospheric Research
 NCAR Command Language Version 6.6.2
 The use of this software is governed by a License Agreement.
 See http://www.ncl.ucar.edu/ for more details.

Variable: pr_aave_in
Type: float
Total Size: 55516 bytes
            13879 values
Number of Dimensions: 1
Dimensions and sizes:	[time | 13879]
Coordinates: 
            time: [47847.375..61725.375]
Number Of Attributes: 14
  _FillValue :	1e+20
  standard_name :	lwe_thickness_of_precipitation_amount
  long_name :	Daily precipitation
  units :	mm
  grid_mapping :	spatial_ref
  least_significant_digit :	3
  cell_methods :	time: sum
  analysis_version_number :	3.01
  number_of_stations_reporting :	6600
  frequency :	daily
  length_scale_for_analysis :	  80
  coverage_content_type :	physicalMeasurement
  missing_value :	1e+20
  wgt_areaave_op_ncl :	Area Average

Variable: pr_aave
Type: float
Total Size: 55516 bytes
            13879 values
Number of D